# Import Bibliotheken

In [2]:
# Festlegung des Device
import platform

# Laden der Daten
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

# Operationen mit Texten
import torchtext
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
import spacy

# Modell
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

# Evaluierung
import torchmetrics

# Auswertung der Evaluation
import pandas as pd
import altair as alt

### Package Versionen

In [4]:
for package in [torchtext,torch, torchmetrics, spacy, alt, pd]:
    print(f'{package.__name__}: {package.__version__}')

torchtext: 0.13.1
torch: 1.12.1
torchmetrics: 0.9.3
spacy: 3.3.1
altair: 4.2.0
pandas: 1.4.3


# Device Auswahl

In [5]:
device = torch.device(
    "cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

# Daten

## Dataset

### Erstellung Dataset

In [6]:
comment_df = pd.read_excel('data/clean/Google_Rezensionen.xlsx')

In [7]:
class TeamGoogleBewertungenDataSet(Dataset):

    def __init__(self, data):
        self.data = data
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        text = comment_df[index]['Kommentar']
        return text

### Bestimmung Test- und Traningsdaten

In [9]:
# Länge der Test- und Traningsdaten bestimmen
dataset = TeamGoogleBewertungenDataSet(comment_df)
len_train = round(len(dataset)* 0.7)
len_test = round(len(dataset) * 0.3)
assert len(comment_df) == len_train + len_test

In [11]:
# Zufällige Aufsplittung der Anzahl von Test- und Trainingsdaten
train_set, test_set = torch.utils.data.random_split(dataset, [len_train, len_test]) 

## Dataloader

### Tokenisierung

In [12]:
tokenizer = get_tokenizer('spacy', language='de_core_news_lg')

c:\Users\flori\anaconda3\envs\ki_env\lib\site-packages\spacy\util.py:837: UserWarning: [W095] Model 'de_core_news_lg' (3.4.0) was trained with spaCy v3.4 and may not be 100% compatible with the current version (3.3.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  version_a (str): The first version


In [14]:
# Beispiel für Anwendung des tokenizer
sentence = 'Sehr nette Bedienung Tolles Angebot Sehr lange nicht mehr so saubere Toiletten gesehen Perfekt Weiter so'
tokenizer(sentence)

['Sehr',
 'nette',
 'Bedienung',
 'Tolles',
 'Angebot',
 'Sehr',
 'lange',
 'nicht',
 'mehr',
 'so',
 'saubere',
 'Toiletten',
 'gesehen',
 'Perfekt',
 'Weiter',
 'so']

### Vokabular erstellen

#### Vokabular wird nicht erstellt !!!

In [60]:
def yield_tokens(data_iter):
    for text, _ in data_iter:
        yield tokenizer(text)

In [57]:
len(train_set)

3533

In [58]:
tokens = yield_tokens(train_set)

In [62]:
vocab = build_vocab_from_iterator(tokens, min_freq=1, specials=["<unk>"])

vocab.set_default_index(vocab["<unk>"])

In [63]:
len(vocab)

1

In [29]:
vocab(['nette', 'Bedienung','saubere'])

[0, 0, 0]

In [32]:
vocab.lookup_token(0)

'<unk>'

### Encoding

In [39]:
onehot = torch.zeros(1,len(vocab))

In [41]:
pos = vocab(['nette', 'Bedienung', 'sauber'])
pos

[0, 0, 0]

In [43]:
def encode(text, vocab):
    tokens = tokenizer(text)
    onehot = torch.zeros(1,len(vocab))
    onehot[:,vocab(tokens)] = 1
    return onehot

### Erstellung Dataloader

In [44]:
label_pipeline = lambda x: int(x) -1

In [45]:
def collate_batch(batch):
    label_list, text_list = [], []
 
    for (_text,_label) in batch:
    
        # Vorverarbeitung der Label
        label_list.append(label_pipeline(_label))
        
        # Vorverarbeitung der Texte
        processed_text = encode(_text, vocab)
        
        # Zusammenführen sämtlicher Textrepräsentationen in einer Liste
        text_list.append(processed_text)
 
    # Zusammenführen aller Label in einem Tensor
    labels = torch.tensor(label_list, dtype=torch.int64)
    
    # Verbinden der Tensoren in text_list zu einem Tensor
    texts = torch.cat(text_list, dim = 0)
    
    # Ausgabe der Texte und der Label
    return texts.to(device), labels.to(device)

In [46]:
train_loader = DataLoader(
 train_set, batch_size=64,
 shuffle=True,
 collate_fn=collate_batch,
 num_workers=0
)